<a href="https://colab.research.google.com/github/Mrsnellek/MSDS_686/blob/22F8W2/MSDS%20686/Week_3/Reuters_Regularization_and_Dropout_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This Example was adapted from Deep Learning with Python Chapters 5 and 7 Chollet, F. (2021). Deep Learning with Python (2nd ed.). Greenwich, CT, USA: Manning Publications Co.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## We will improve our previous Reuters neural net by adjusting learning rate, and adding dropout, and early stopping.

In [ ]:
from tensorflow.keras.datasets import reuters
import numpy as np
np.random.seed(1)
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)
# When we import the data we only select the 10,000 most common words in the Reuters dataset.

## Previous Baseline Model

In [ ]:
# Remember, we cannot add a single vector to a neural network. The data needs to be converted to a tensor.  This function will create a tenor that is 25000 by 10000. 25000 is the number of samples
# and 10000 is the number of uniqe words. The tensor will have all zeros except for ones where that word is in the rewview
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [ ]:
# Apply the vectorize function to the train_data and test_data
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
# Convert the y targets to categorical
from keras.utils.np_utils import to_categorical
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

In [ ]:
# Split the data into training and validataion sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
# Import the keras libraries
from tensorflow import keras
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend

In [ ]:
# Build the model architecture.  Input shape must equal the number of vectors or (unique words).
# Add one hidden layer with 64 nodes.  Keep the activation function as 'relu'
# Since we have a multiclass classification, the output activation function will be 'softmax'
# We will keep with the 'rmsprop' optomizer function, loss = 'categorical_crossentropy', and metrics = 'accuracy'
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))
model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
# Fit the model to the training data. 
baseline_history = model.fit(x_train,
                   y_train,
                   epochs = 50,
                   batch_size = 512,
                   validation_data = (x_val, y_val))

In [ ]:
baseline_history = pd.DataFrame(baseline_history.history)
print(baseline_history)

In [ ]:
# Let's plot the Loss vs Epochs and Accuracy vs Epochs
baseline_loss_values = baseline_history['loss']
baseline_val_loss_values = baseline_history['val_loss']
baseline_acc_values = baseline_history['accuracy']
baseline_val_acc_values = baseline_history['val_accuracy']
epochs = range(1, len(baseline_history['accuracy']) + 1)

In [ ]:
plt.plot(epochs, baseline_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, baseline_val_loss_values, 'b', label = 'Baseline Validation loss')
plt.title('Baseline Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Baseline Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, baseline_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, baseline_val_acc_values, 'b', label = 'Baseline Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Baseline Accuracy')
plt.legend()
plt.show()

## How does learning rate affect accuracy?
RMSprop default learning rate = 0.001. https://keras.io/api/optimizers/rmsprop/ 

In [ ]:
# Change the learning rate to 1
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(1),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

lr_history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 500,
              validation_data = (x_val, y_val))

history_dict = lr_history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, baseline_val_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Baseline and current and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, baseline_val_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

Not very good.  The learning rate is too high and cannot converge on a decent accuracy. What happens if we lower the learning to 0.0005?

In [ ]:
# Change the learning rate to 0.0005
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

lr_history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 500,
              validation_data = (x_val, y_val))

history_dict = lr_history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, baseline_val_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Baseline and current and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, baseline_val_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

Lowering the learning rate to 0.005 creates a nice loss and accuracy curve that keeps improving.

## Using a learning_rate = 0.0005 let us adjust the batchsize.

In [ ]:
# Lower batch size to 128
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 128,
              validation_data = (x_val, y_val),
              verbose = 2)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, baseline_val_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Baseline and current and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, baseline_val_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

In [ ]:
# Increase batchsize to 1024
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 1024,
              validation_data = (x_val, y_val))

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, baseline_val_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Baseline and current and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, baseline_val_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

## It looks like the large batch size imporves the accuracy over the baseline. 


## Keeping the 0.0005 learning rate and 1024 batch size let's add drop out.

In [ ]:
# Add dropout
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 1024,
              validation_data = (x_val, y_val),
              verbose = 2)

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, baseline_val_loss_values, 'bo', label = 'Baseline Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Baseline and current and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, baseline_val_acc_values, 'bo', label = 'Baseline Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Baseline Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

## We keep improving.



In [ ]:
# Add Early Stoping
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(x_train,
              y_train,
              epochs = 50,
              batch_size = 1024,
              validation_data = (x_val, y_val),
              verbose = 2,
              callbacks=[EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights = True)])

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, val_loss_values, 'bo', label = 'Validation loss')
plt.title('Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, val_acc_values, 'bo', label = 'Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)

In [ ]:
# Make a better, deeper model.
backend.clear_session()
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_shape = (10000,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(46, activation = 'softmax'))

model.compile(optimizer=keras.optimizers.RMSprop(0.0005),
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

history = model.fit(x_train,
              y_train,
              epochs = 100,
              batch_size = 1024,
              validation_data = (x_val, y_val),
              verbose = 2,
              callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights = True)])

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, val_loss_values, 'bo', label = 'Validation loss')
plt.title('Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, val_acc_values, 'bo', label = 'Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

results = model.evaluate(x_test, y_test)
print(model.metrics_names)
print(results)